In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_table('data\\toy.txt', names=['feature_1', 'feature_2', 'target'])
df.head()

,feature_1,feature_2,target
0,0.77,-1.14,0
1,-0.33,1.44,0
2,0.91,-3.07,0
3,-0.37,-1.91,0
4,-1.84,-1.13,0


In [3]:
# determine whether system supports CUDA or not 
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
# prepare data (training & testing)
X = df[['feature_1', 'feature_2']]
y = df['target'] 

# split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# convert numpy arrays into pytorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float, device=DEVICE)
y_train = torch.tensor(y_train.values, dtype=torch.float, device=DEVICE)
X_test = torch.tensor(X_test.values, dtype=torch.float, device=DEVICE)
y_test = torch.tensor(y_test.values, dtype=torch.float, device=DEVICE)

## On-line mode

In [5]:
class Perceptron():
    # initialize the number of features, weights and bias
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = torch.zeros(size=(num_features, 1), dtype=torch.float32, device=DEVICE)
        self.bias = torch.zeros(size=(1, ), dtype=torch.float32, device=DEVICE)
        
    # wx + b (1)
    def forward(self, x):
        linear = torch.add(torch.mm(x, self.weights), self.bias)
        predictions = torch.where(linear > 0., 1, 0)
        return predictions
    
    # compute errors (2)
    def backward(self, x, y):
        predictions = self.forward(x).reshape(-1)
        errors = y - predictions
        return errors

    # for every epoch implement (1) + (2) and update errors
    def train(self, x, y, num_epochs):
        for e in range(num_epochs):
            for i in range(len(y)):
                errors = self.backward(x[i].view(1, self.num_features), y[i]).reshape(-1)
                self.weights = self.weights + (x[i]*errors).reshape(self.num_features, 1)
                self.bias = self.bias + errors
    
            ### Logging ###
            print('Epoch: %03d' %(e+1), end='')
            print(' | Accuracy: %.3f' %self.evaluate(x, y), end='')
            print(' | Error %.3f' % self.bias)
            
    # define 'evaluate' method
    def evaluate(self, x, y):
        predictions = self.forward(x).reshape(-1)
        accuracy = torch.sum(predictions==y) / y.size(0)
        return accuracy

In [6]:
model = Perceptron(num_features=X_train.size(1))
model.train(X_train, y_train, num_epochs=20)

Epoch: 001 | Accuracy: 1.000 | Error -2.000
Epoch: 002 | Accuracy: 1.000 | Error -2.000
Epoch: 003 | Accuracy: 1.000 | Error -2.000
Epoch: 004 | Accuracy: 1.000 | Error -2.000
Epoch: 005 | Accuracy: 1.000 | Error -2.000
Epoch: 006 | Accuracy: 1.000 | Error -2.000
Epoch: 007 | Accuracy: 1.000 | Error -2.000
Epoch: 008 | Accuracy: 1.000 | Error -2.000
Epoch: 009 | Accuracy: 1.000 | Error -2.000
Epoch: 010 | Accuracy: 1.000 | Error -2.000
Epoch: 011 | Accuracy: 1.000 | Error -2.000
Epoch: 012 | Accuracy: 1.000 | Error -2.000
Epoch: 013 | Accuracy: 1.000 | Error -2.000
Epoch: 014 | Accuracy: 1.000 | Error -2.000
Epoch: 015 | Accuracy: 1.000 | Error -2.000
Epoch: 016 | Accuracy: 1.000 | Error -2.000
Epoch: 017 | Accuracy: 1.000 | Error -2.000
Epoch: 018 | Accuracy: 1.000 | Error -2.000
Epoch: 019 | Accuracy: 1.000 | Error -2.000
Epoch: 020 | Accuracy: 1.000 | Error -2.000


In [7]:
print(f'Test Accuracy {model.evaluate(X_test, y_test):.3f}')

Test Accuracy 1.000


## Minibatch-mode

In [8]:
class Perceptron():
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = torch.zeros(num_features, 1, dtype=torch.float32, device=DEVICE)
        self.bias = torch.zeros(1, dtype=torch.float32, device=DEVICE)
        
    def forward(self, x):
        linear = torch.mm(x, self.weights) + self.bias
        predictions = torch.where(linear > 0., 1, 0)
        return predictions.view(-1)
    
    def backward(self, x, y):
        predictions = self.forward(x)
        errors = y - predictions
        return errors
    
    def train(self, x, y, num_epochs, minibatch_size=10, seed=123):
        shuffle_idx = torch.randperm(y.size(0))
        minibatches = torch.split(shuffle_idx, minibatch_size)
        torch.manual_seed(seed)
        
        for e in range(num_epochs):
            for minibatch_idx in minibatches:
                errors = self.backward(x[minibatch_idx], y[minibatch_idx])
                self.weights += torch.mm(errors.view(-1, 1).t(), x[minibatch_idx]).view(self.num_features, 1)
                self.bias = torch.sum(errors)

            ### Logging ###
            print('Epoch: %03d' %(e+1), end='')
            print(' | Accuracy: %.3f' %self.evaluate(x, y), end='')
            print(' | Error: %.3f' % self.bias)

    def evaluate(self, x, y):
        y_pred = self.forward(x)
        accuracy = torch.sum(y_pred==y) / y.size(0)
        return accuracy

In [9]:
model = Perceptron(num_features=X_train.size(1))
model.train(X_train, y_train, num_epochs=20)

Epoch: 001 | Accuracy: 0.960 | Error: 0.000
Epoch: 002 | Accuracy: 0.960 | Error: 0.000
Epoch: 003 | Accuracy: 0.960 | Error: 0.000
Epoch: 004 | Accuracy: 0.960 | Error: 0.000
Epoch: 005 | Accuracy: 0.960 | Error: 0.000
Epoch: 006 | Accuracy: 0.960 | Error: 0.000
Epoch: 007 | Accuracy: 0.960 | Error: 0.000
Epoch: 008 | Accuracy: 0.960 | Error: 0.000
Epoch: 009 | Accuracy: 0.960 | Error: 0.000
Epoch: 010 | Accuracy: 0.960 | Error: 0.000
Epoch: 011 | Accuracy: 0.960 | Error: 0.000
Epoch: 012 | Accuracy: 0.973 | Error: 0.000
Epoch: 013 | Accuracy: 0.960 | Error: 0.000
Epoch: 014 | Accuracy: 0.960 | Error: 0.000
Epoch: 015 | Accuracy: 0.960 | Error: 0.000
Epoch: 016 | Accuracy: 0.960 | Error: 0.000
Epoch: 017 | Accuracy: 0.973 | Error: 0.000
Epoch: 018 | Accuracy: 0.973 | Error: 0.000
Epoch: 019 | Accuracy: 0.973 | Error: 0.000
Epoch: 020 | Accuracy: 0.973 | Error: 0.000


In [10]:
print(f'Test Accuracy: {model.evaluate(X_test, y_test):.3f}')

Test Accuracy: 0.960


## Bacth mode

In [11]:
class Perceptron():
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = torch.zeros(num_features, 1, dtype=torch.float32, device=DEVICE)
        self.bias = torch.zeros(1, dtype=torch.float32, device=DEVICE)
        
    def forward(self, x):
        linear = torch.mm(x, self.weights) + self.bias
        predictions = torch.where(linear > 0., 1, 0)
        return predictions.view(-1)
    
    def backward(self, x, y):
        predictions = self.forward(x)
        errors = y - predictions
        return errors
    
    def train(self, x, y, num_epochs):
        for e in range(num_epochs):
            errors = self.backward(x, y)
            self.weights += torch.mm(errors.view(-1, 1).t(), x).view(self.num_features, 1)
            self.bias = torch.sum(errors)
            
            ### Logging ###
            print('Epoch: %03d' %(e+1), end='')
            print(' | Accuracy: %.3f' %self.evaluate(x, y), end='')
            print(' | Error: %.3f' % self.bias)
    
    def evaluate(self, x, y):
        y_pred = self.forward(x)
        accuracy = torch.sum(y_pred==y) / y.size(0)
        return accuracy

In [12]:
model = Perceptron(num_features=X_train.size(1))
model.train(X_train, y_train, num_epochs=20)

Epoch: 001 | Accuracy: 0.960 | Error: 37.000
Epoch: 002 | Accuracy: 0.973 | Error: -3.000
Epoch: 003 | Accuracy: 0.973 | Error: -2.000
Epoch: 004 | Accuracy: 0.973 | Error: -2.000
Epoch: 005 | Accuracy: 0.973 | Error: -2.000
Epoch: 006 | Accuracy: 0.973 | Error: -2.000
Epoch: 007 | Accuracy: 0.973 | Error: -2.000
Epoch: 008 | Accuracy: 0.973 | Error: -2.000
Epoch: 009 | Accuracy: 0.973 | Error: -2.000
Epoch: 010 | Accuracy: 0.973 | Error: -2.000
Epoch: 011 | Accuracy: 0.973 | Error: -2.000
Epoch: 012 | Accuracy: 0.973 | Error: -2.000
Epoch: 013 | Accuracy: 0.973 | Error: -2.000
Epoch: 014 | Accuracy: 0.973 | Error: -2.000
Epoch: 015 | Accuracy: 0.973 | Error: -2.000
Epoch: 016 | Accuracy: 0.973 | Error: -2.000
Epoch: 017 | Accuracy: 0.973 | Error: -2.000
Epoch: 018 | Accuracy: 0.973 | Error: -2.000
Epoch: 019 | Accuracy: 0.973 | Error: -2.000
Epoch: 020 | Accuracy: 0.973 | Error: -2.000


In [13]:
print(f'Test Accuracy: {model.evaluate(X_test, y_test):.3f}')

Test Accuracy: 0.960
